# DuckDB overview

# DuckDB on Datalore

DuckDB is a resource-efficient analytical database you can use in your Datalore notebooks. Designed for columnar, vectorized processing, it lets you run efficient SQL queries on large datasets—directly within your notebooks.

DuckDB is a feature-rich analytical SQL engine that supports a full SQL dialect and a variety of file formats, such as CSV, Parquet, and JSON—allowing you to work directly on your raw data. It also includes additional functions and optimizations, such as advanced aggregate and window functions, designed specifically for fast, in-memory analytics. These functions enable precise summarizations and detailed row-level comparisons without the need for additional processing steps. DuckDB's in-memory, columnar engine delivers quick query results, efficiently handling complex operations and larger-than-memory workloads.

Follow this notebook for a quick overview of DuckDB's capabilities, or check the [DuckDB documentation](https://duckdb.org/why_duckdb) for in-depth insights.


## Read **CSV, JSON, and Parquet** files on the fly

DuckDB provides **built-in** functions, such as [`read_csv_auto`](https://duckdb.org/docs/data/csv/overview.html), which **let** you query CSV files **on the fly** without any **pre-importing**. You can also leverage DuckDB to read **other file types**, such as JSON or **Parquet**, using the [`read_parquet`](https://duckdb.org/docs/data/parquet/overview) and [`read_json`](https://duckdb.org/docs/data/json/overview) functions, respectively. Using these functions, you don't need to load the data into your tables. On top of that, DuckDB **allows** you to combine [multiple file formats](https://duckdb.org/docs/data/multiple_files/overview) in your queries.

The example below shows **the usage of the** `read_csv_auto` function.

In [1]:
DROP TABLE IF EXISTS sample_data;

CREATE TABLE sample_data AS
SELECT *
FROM read_csv_auto('sample.csv');

-- Select all data from the table to see the contents
SELECT * FROM sample_data;

id 
 name 
 age 
 city 
 
 
 
 
 0 
 1 
 Alice 
 30 
 New York 
 
 
 1 
 2 
 Bob 
 25 
 Los Angeles 
 
 
 2 
 3 
 Charlie 
 35 
 Chicago 
 
 
 3 
 4 
 Dave 
 28 
 Houston

## Query large datasets efficiently

When working with massive datasets, computing the exact distinct count using standard SQL functions such as `COUNT(DISTINCT ...)` can be resource-intensive and slow. 

DuckDB's [`approx_count_distinct`](https://duckdb.org/docs/sql/functions/aggregates.html#approximate-aggregates) function provides a fast, memory-efficient way to estimate the number of unique values with a high degree of accuracy. This means you can quickly gain insights into your data’s uniqueness over large datasets without having to wait for long-running queries. These functions can be especially useful in exploratory analytics, where speed is crucial.

Check [this document](https://duckdb.org/docs/sql/functions/overview) for an overview of other DuckDB functions built for resource-efficient querying of your data.

In [2]:
DROP TABLE IF EXISTS sample_numbers;

CREATE TABLE sample_numbers AS
SELECT *
FROM read_csv_auto('data.csv');

-- Compute the exact count of distinct values
SELECT COUNT(DISTINCT value) AS exact_distinct_values
FROM sample_numbers;

-- Compute the approximate count of distinct values using approx_count_distinct
SELECT approx_count_distinct(value) AS approx_distinct_values
FROM sample_numbers;

approx_distinct_values 
 
 
 
 
 0 
 11

## Advanced analytics with window functions

DuckDB also supports advanced SQL analytics, including window functions such as `ROW_NUMBER()` and `RANK()`. Using them, you can perform calculations over rows that are correlated to the current row.

For instance, you can rank rows based on a specific metric. You can also calculate running totals or compute moving averages directly in your SQL queries. Using DuckDB's window functions, you can simplify your advanced analytics as you'll no longer need to perform multiple steps in your analyses or use external tools. Using them, you can, for example, see how a particular metric trends over time or compare an individual row against its peers with a single query.

In [3]:
DROP TABLE IF EXISTS my_table;

CREATE TABLE my_table (
    name VARCHAR,
    value INTEGER
);

INSERT INTO my_table (name, value)
VALUES
    ('Alice', 50),
    ('Bob', 70),
    ('Charlie', 60),
    ('Dave', 90);

-- Use a window function to assign a rank based on the "value" column (highest value gets rank 1)
SELECT 
    name,
    value,
    row_number() OVER (ORDER BY value DESC) AS rank
FROM my_table;


name 
 value 
 rank 
 
 
 
 
 0 
 Dave 
 90 
 1 
 
 
 1 
 Bob 
 70 
 2 
 
 
 2 
 Charlie 
 60 
 3 
 
 
 3 
 Alice 
 50 
 4

In [4]:
-- --------------------------------------------------
-- Oil Prices by Geo: Creating a table and computing statistics
-- --------------------------------------------------

DROP TABLE IF EXISTS oil_prices;

CREATE TABLE oil_prices (
    id INTEGER,
    geo VARCHAR,
    price DOUBLE
);

INSERT INTO oil_prices VALUES 
(1, 'USA', 70.5),
(2, 'USA', 68.2),
(3, 'Canada', 72.1),
(4, 'Russia', 65.0),
(5, 'Saudi Arabia', 67.8),
(6, 'Iran', NULL),
(7, 'Venezuela', 60.0),
(8, 'Norway', 74.0),
(9, 'Mexico', 69.5),
(10, 'USA', 71.2),
(11, 'Nigeria', 80.0),
(12, 'Nigeria', 82.3),
(13, 'Brazil', 64.5),
(14, 'Brazil', 63.8),
(15, 'USA', 120.0);

DESCRIBE oil_prices;

SUMMARIZE oil_prices;



column_name 
 column_type 
 min 
 max 
 approx_unique 
 avg 
 std 
 q25 
 q50 
 q75 
 count 
 null_percentage 
 
 
 
 
 0 
 id 
 INTEGER 
 1 
 15 
 17 
 8.0 
 4.47213595499958 
 4 
 8 
 12 
 15 
 0.00 
 
 
 1 
 geo 
 VARCHAR 
 Brazil 
 Venezuela 
 11 
 None 
 None 
 None 
 None 
 None 
 15 
 0.00 
 
 
 2 
 price 
 DOUBLE 
 60.0 
 120.0 
 15 
 73.49285714285713 
 14.688062357716753 
 65.0 
 70.0 
 74.0 
 15 
 6.67

## Aggregate functions

DuckDB aggregate functions like `SUM()`, `AVG()`, `MIN()`, and `MAX()` allow you to quickly compute totals, averages, or extremes over large datasets. Because DuckDB is designed for columnar, vectorized processing, these operations are performed efficiently.

You can also use them for grouping and comparative analyses. For example, when combined with the `GROUP BY` clause, aggregate functions let you slice your data by various dimensions (such as geographic region, time period, or product category) to compare metrics side by side. This allows you to identify trends, outliers, or patterns in your data quickly.

On top of that, you can also combine DuckDB’s aggregate functions with window functions. This means you can not only compute group-level summaries but also compare each row against these summaries (for example, calculating a moving average or ranking items within a group), all in a single query.

In [5]:

DROP TABLE IF EXISTS oil_prices;

CREATE TABLE oil_prices (
    id INTEGER,
    geo VARCHAR,
    price DOUBLE
);

INSERT INTO oil_prices VALUES 
(1, 'USA', 70.5),
(2, 'USA', 68.2),
(3, 'Canada', 72.1),
(4, 'Russia', 65.0),
(5, 'Saudi Arabia', 67.8),
(6, 'Iran', NULL),
(7, 'Venezuela', 60.0),
(8, 'Norway', 74.0),
(9, 'Mexico', 69.5),
(10, 'USA', 71.2),
(11, 'Nigeria', 80.0),
(12, 'Nigeria', 82.3),
(13, 'Brazil', 64.5),
(14, 'Brazil', 63.8),
(15, 'USA', 120.0);

WITH geo_stats AS (
    SELECT 
        geo,
        COUNT(*) AS total_count,
        AVG(price) AS avg_price,
        MIN(price) AS min_price,
        MAX(price) AS max_price
    FROM oil_prices
    GROUP BY geo
)

SELECT 
    o.id,
    o.geo,
    o.price,
    gs.total_count,
    gs.avg_price,
    gs.min_price,
    gs.max_price,
    -- Calculate the deviation of each price from its region's average
    o.price - gs.avg_price AS deviation,
    -- Rank each row within its geographic group by price (highest price gets rank 1)
    ROW_NUMBER() OVER (PARTITION BY o.geo ORDER BY o.price DESC) AS rank_within_geo
FROM oil_prices o
JOIN geo_stats gs ON o.geo = gs.geo
ORDER BY o.geo, rank_within_geo;




id 
 geo 
 price 
 total_count 
 avg_price 
 min_price 
 max_price 
 deviation 
 rank_within_geo 
 
 
 
 
 0 
 13 
 Brazil 
 64.5 
 2 
 64.150 
 63.8 
 64.5 
 0.350 
 1 
 
 
 1 
 14 
 Brazil 
 63.8 
 2 
 64.150 
 63.8 
 64.5 
 -0.350 
 2 
 
 
 2 
 3 
 Canada 
 72.1 
 1 
 72.100 
 72.1 
 72.1 
 0.000 
 1 
 
 
 3 
 6 
 Iran 
 NaN 
 1 
 NaN 
 NaN 
 NaN 
 NaN 
 1 
 
 
 4 
 9 
 Mexico 
 69.5 
 1 
 69.500 
 69.5 
 69.5 
 0.000 
 1 
 
 
 5 
 12 
 Nigeria 
 82.3 
 2 
 81.150 
 80.0 
 82.3 
 1.150 
 1 
 
 
 6 
 11 
 Nigeria 
 80.0 
 2 
 81.150 
 80.0 
 82.3 
 -1.150 
 2 
 
 
 7 
 8 
 Norway 
 74.0 
 1 
 74.000 
 74.0 
 74.0 
 0.000 
 1 
 
 
 8 
 4 
 Russia 
 65.0 
 1 
 65.000 
 65.0 
 65.0 
 0.000 
 1 
 
 
 9 
 5 
 Saudi Arabia 
 67.8 
 1 
 67.800 
 67.8 
 67.8 
 0.000 
 1 
 
 
 10 
 15 
 USA 
 120.0 
 4 
 82.475 
 68.2 
 120.0 
 37.525 
 1 
 
 
 11 
 10 
 USA 
 71.2 
 4 
 82.475 
 68.2 
 120.0 
 -11.275 
 2 
 
 
 12 
 1 
 USA 
 70.5 
 4 
 82.475 
 68.2 
 120.0 
 -11.975 
 3 
 
 
 13 
 2 
 USA 
 68.2 
 4 
 82.475 
 68.2 
 120.0 
 -14.275 
 4 
 
 
 14 
 7 
 Venezuela 
 60.0 
 1 
 60.000 
 60.0 
 60.0 
 0.000 
 1

## Overview of all DuckDB aggregate functions

This query displays a unique list of DuckDB aggregate functions starting with the letter "A." It pulls details like the function name, type, return type, parameters, and description from the built-in metadata. The `DISTINCT` clause ensures only one entry per function is shown, and the results are ordered alphabetically.

Feel free to change the pattern in the `WHERE` clause (for example, to `LIKE 'a%'` or `LIKE 'b%'`) and run it again to explore DuckDB's wide range of functions starting with any letter.

In [6]:
SELECT DISTINCT ON (function_name)
    function_name,
    function_type,
    return_type,
    parameters,
    parameter_types,
    description
FROM duckdb_functions()
WHERE function_type = 'aggregate'
  AND function_name LIKE 'a%'
ORDER BY function_name;



function_name 
 function_type 
 return_type 
 parameters 
 parameter_types 
 description 
 
 
 
 
 0 
 any_value 
 aggregate 
 DECIMAL 
 [col0] 
 [DECIMAL] 
 None 
 
 
 1 
 approx_count_distinct 
 aggregate 
 BIGINT 
 [any] 
 [ANY] 
 Computes the approximate count of distinct ele... 
 
 
 2 
 approx_quantile 
 aggregate 
 DECIMAL 
 [x, pos] 
 [DECIMAL, FLOAT] 
 Computes the approximate quantile using T-Digest. 
 
 
 3 
 approx_top_k 
 aggregate 
 ANY[] 
 [val, k] 
 [ANY, BIGINT] 
 Finds the k approximately most occurring value... 
 
 
 4 
 arbitrary 
 aggregate 
 DECIMAL 
 [col0] 
 [DECIMAL] 
 None 
 
 
 5 
 arg_max 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the maximum val. Calculates... 
 
 
 6 
 arg_max_null 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the maximum val. Calculates... 
 
 
 7 
 arg_min 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the minimum val. Calculates... 
 
 
 8 
 arg_min_null 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the minimum val. Calculates... 
 
 
 9 
 argmax 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the maximum val. Calculates... 
 
 
 10 
 argmin 
 aggregate 
 INTEGER 
 [arg, val] 
 [INTEGER, INTEGER] 
 Finds the row with the minimum val. Calculates... 
 
 
 11 
 array_agg 
 aggregate 
 LIST 
 [arg] 
 [ANY] 
 Returns a LIST containing all the values of a ... 
 
 
 12 
 avg 
 aggregate 
 DECIMAL 
 [x] 
 [DECIMAL] 
 Calculates the average value for all tuples in x.

In [7]:
import pandas as pd


data = {
    "Feature": [
        "Parquet Files",
        "CSV Files",
        "JSON Files",
        "Window Functions",
        "Scalar Functions",
        "Aggregate Functions",
        "Python Integration",
        "SQLite Integration"
    ],
    "Description": [
        "Query Parquet files directly without pre-loading data.",
        "Read CSV files with automatic schema inference.",
        "Query JSON files including nested structures.",
        "Use advanced window functions like row_number, rank, etc.",
        "Transform data with a wide range of scalar functions.",
        "Summarize data using functions like avg, sum, count, etc.",
        "Integrate with Pandas DataFrames and run SQL on Python data.",
        "Directly query SQLite databases from DuckDB."
    ],
    "Example Usage": [
        "SELECT * FROM read_parquet('data/sample.parquet');",
        "SELECT * FROM read_csv_auto('data/sample.csv');",
        "SELECT * FROM read_json('data/sample.json');",
        "SELECT col, ROW_NUMBER() OVER (ORDER BY col) AS rn FROM my_table;",
        "SELECT ABS(-5);",
        "SELECT AVG(price) FROM oil_prices;",
        "import duckdb\nduckdb.query('SELECT * FROM df').to_df();",
        "SELECT * FROM sqlite_scan('db.sqlite', 'table');"
    ]
}

# Create the DataFrame
duckdb_features_df = pd.DataFrame(data)

# Display the DataFrame
duckdb_features_df

Feature 
 Description 
 Example Usage 
 
 
 
 
 0 
 Parquet Files 
 Query Parquet files directly without pre-loadi... 
 SELECT * FROM read_parquet('data/sample.parque... 
 
 
 1 
 CSV Files 
 Read CSV files with automatic schema inference. 
 SELECT * FROM read_csv_auto('data/sample.csv'); 
 
 
 2 
 JSON Files 
 Query JSON files including nested structures. 
 SELECT * FROM read_json('data/sample.json'); 
 
 
 3 
 Window Functions 
 Use advanced window functions like row_number,... 
 SELECT col, ROW_NUMBER() OVER (ORDER BY col) A... 
 
 
 4 
 Scalar Functions 
 Transform data with a wide range of scalar fun... 
 SELECT ABS(-5); 
 
 
 5 
 Aggregate Functions 
 Summarize data using functions like avg, sum, ... 
 SELECT AVG(price) FROM oil_prices; 
 
 
 6 
 Python Integration 
 Integrate with Pandas DataFrames and run SQL o... 
 import duckdb\nduckdb.query('SELECT * FROM df'... 
 
 
 7 
 SQLite Integration 
 Directly query SQLite databases from DuckDB. 
 SELECT * FROM sqlite_scan('db.sqlite', 'table');